In [7]:
import cv2
import os
import xml.etree.ElementTree as ET
from tkinter import filedialog

def extract_frames(xml_path, video_path, output_folder):
    # XML 파일 파싱
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 동영상 파일 열기
    cap = cv2.VideoCapture(video_path)
    file_name = root.find('.//filename').text[:-4]
    event_name = root.find('.//eventname').text
    i = 1
    # 프레임 추출 및 저장
    for action in root.findall('.//action'):

        for frame_info in action.findall('.//frame'):
            start_frame = int(frame_info.find('start').text)
            end_frame = int(frame_info.find('end').text)
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, (start_frame+end_frame)//2)
            ret, frame = cap.read()
            
            # 이미지 저장
            image_path = os.path.join(output_folder, f"{event_name}_{i}_{file_name}.jpg")
            i += 1
            cv2.imwrite(image_path, frame)

    # 비디오 파일 닫기
    cap.release()

# XML 파일 경로, 동영상 파일 경로 및 결과 이미지 저장 폴더 지정
#xml_path = './data/cctv/swoon/710-3_cam01_swoon01_place01_day_spring.xml'
#video_path = './data/cctv/swoon/710-3_cam01_swoon01_place01_day_spring.mp4'
#output_folder = './data/output_images/'

# 결과 이미지 저장 폴더 생성
#os.makedirs(output_folder, exist_ok=True)

# 프레임 추출 및 저장
#extract_frames(xml_path, video_path, output_folder)


In [8]:
# CCTV 영상 데이터 폴더 경로 지정
data_path = filedialog.askdirectory(initialdir="./data", title="cctv video data folder path")
output_folder = ""

for folder_name in os.listdir(data_path):
    data_folder_path = os.path.join(data_path, folder_name)
    
    for idx, data_folder_name in enumerate(os.listdir(data_folder_path)):
        folder_path = os.path.join(data_folder_path, data_folder_name)
        
        # 이미지 저장 경로 지정
        if not output_folder:
            output_folder = filedialog.askdirectory(initialdir="./data", title="output image folder path")

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            if os.path.isfile(file_path) and file_name.lower().endswith('.xml'):
                xml_path = file_path
                video_path = os.path.splitext(xml_path)[0] + '.mp4'

                # 결과 이미지 저장 폴더 생성
                os.makedirs(output_folder, exist_ok=True)

                extract_frames(xml_path, video_path, output_folder)